# MTCNN

In [1]:
import json
import os
import cv2
from mtcnn import MTCNN
import numpy as np
from tqdm import tqdm 
import logging

#from evaluation_utils import evaluate_predictions

In [2]:
# Configure logging
logging.basicConfig(
    filename='mtcnn_evaluation.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger()


In [3]:
def calc_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2]) 
    y2 = min(box1[3], box2[3]) 

    intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    union = box1_area + box2_area - intersection

    return intersection / union if union > 0 else 0

def calc_precision(true_positives, false_positives):
    if true_positives + false_positives > 0:
        return true_positives / (true_positives + false_positives)
    else:
        return 0

def calc_recall(true_positives, false_negatives):
    if true_positives + false_negatives > 0:
        return true_positives / (true_positives + false_negatives)
    else:
        return 0

def calc_f1_score(precision, recall):
    if precision + recall > 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0

def evaluate_predictions(predictions, ground_truth, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    matched = [False] * len(ground_truth)

    for pred in predictions:
        matched_any = False
        for i, gt in enumerate(ground_truth):
            if not matched[i] and calc_iou(pred, gt) >= iou_threshold:
                matched[i] = True
                matched_any = True
                true_positives += 1
                break
        if not matched_any:
            false_positives += 1
    false_negatives = len(ground_truth) - sum(matched)

    precision = calc_precision(true_positives, false_positives)
    recall = calc_recall(true_positives, false_negatives)
    f1_score = calc_f1_score(precision, recall)

    return precision, recall, f1_score


In [4]:
def evaluate_mtcnn_with_predictions(json_path, output_metrics_path, predictions_output_path, iou_threshold=0.5):
    """
    Evaluate MTCNN model on a subset of images from the given JSON annotation file,
    and save the model's predictions in a separate JSON file.

    Args:
        json_path (str): Path to the JSON file containing annotations.
        output_metrics_path (str): Path to save the evaluation metrics as a JSON file.
        predictions_output_path (str): Path to save the model predictions as a JSON file.
        iou_threshold (float): IoU threshold to consider a prediction as true positive.

    Returns:
        None: Prints mean precision, recall, and F1-score, saves metrics and predictions to JSON files.
    """
    # Load the annotations
    with open(json_path, 'r') as json_file:
        annotations = json.load(json_file)

    # Initialize the MTCNN model
    detector = MTCNN()
    logger.info("Initialized MTCNN detector.")

    # Metrics for all images
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Store predictions
    predictions_data = []

    # Iterate over all image annotations with a progress bar
    for index, image_annotation in enumerate(tqdm(annotations, desc="Evaluating Images")):
        # Extract ground truths and convert to [x1, y1, x2, y2] format
        ground_truths = []
        for gt in image_annotation.get('image_info', []):
            bbox = gt.get('bbox', [])
            if len(bbox) != 4:
                logger.warning(f"Invalid bbox format in image {image_annotation.get('image_path', 'unknown')}. Skipping this ground truth.")
                continue
            x, y, w, h = bbox
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            # Ensure coordinates are positive
            x1, y1, x2, y2 = max(0, x1), max(0, y1), max(0, x2), max(0, y2)
            ground_truths.append([x1, y1, x2, y2])

        # Load the image
        image_path = image_annotation.get('image_path', '')
        if not image_path:
            logger.warning(f"No image path provided for annotation index {index}. Skipping.")
            continue

        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Error: Could not load image at {image_path}. Skipping.")
            continue

        # Convert image from BGR (OpenCV format) to RGB (MTCNN expects RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get predictions using MTCNN
        detections = detector.detect_faces(image_rgb)
        predictions = []
        for face in detections:
            # MTCNN returns 'box' as [x, y, width, height]
            box = face.get('box', [])
            if len(box) != 4:
                logger.warning(f"Invalid detection box format in image {image_path}. Skipping this detection.")
                continue
            x, y, w, h = box
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            # Ensure coordinates are within image boundaries
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(image.shape[1] - 1, x2), min(image.shape[0] - 1, y2)
            predictions.append([x1, y1, x2, y2])

        # Save predictions for this image
        predictions_data.append({
            "image_path": image_path,
            "predicted_boxes": predictions
        })

        # Calculate metrics
        if not predictions and not ground_truths:
            precision, recall, f1_score = 0, 0, 0
        elif not predictions:
            precision, recall, f1_score = 0, 0, 0
        elif not ground_truths:
            precision, recall, f1_score = 0, 0, 0
        else:
            precision, recall, f1_score = evaluate_predictions(predictions, ground_truths, iou_threshold)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1_score)

    # Calculate mean metrics
    mean_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    mean_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
    mean_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0

    # Print the results
    logger.info(f"Mean Precision: {mean_precision:.4f}")
    logger.info(f"Mean Recall: {mean_recall:.4f}")
    logger.info(f"Mean F1-Score: {mean_f1:.4f}")

    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"Mean F1-Score: {mean_f1:.4f}")

    # Save metrics to a JSON file
    output_data = {
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "mean_f1_score": mean_f1,
        "individual_scores": {
            "precision_scores": [float(p) for p in precision_scores],
            "recall_scores": [float(r) for r in recall_scores],
            "f1_scores": [float(f) for f in f1_scores]
        },
        "predictions": predictions_data
    }

    try:
        with open(output_metrics_path, 'w') as output_file:
            json.dump(output_data, output_file, indent=4)
        logger.info(f"Metrics saved to {output_metrics_path}")
        print(f"Metrics saved to {output_metrics_path}")
    except Exception as e:
        logger.error(f"Error saving metrics to {output_metrics_path}: {e}")
        print(f"Error saving metrics to {output_metrics_path}: {e}")

    # Save predictions to a separate JSON file
    try:
        with open(predictions_output_path, 'w') as pred_file:
            json.dump(predictions_data, pred_file, indent=4)
        logger.info(f"Predictions saved to {predictions_output_path}")
        print(f"Predictions saved to {predictions_output_path}")
    except Exception as e:
        logger.error(f"Error saving predictions to {predictions_output_path}: {e}")
        print(f"Error saving predictions to {predictions_output_path}: {e}")

In [11]:
json_path = "Data/WIDER FACE Validation Set/wider_face_num_faces_easy.json"
output_path_metrics = "Outputs/metrics_easy_num_faces_mtcnn.json"
output_path_predictions = "Outputs/predictions_easy_num_faces_mtcnn.json"

evaluate_mtcnn_with_predictions(json_path, output_path_metrics, output_path_predictions)

Evaluating Images: 100%|███| 1122/1122 [10:18<00:00,  1.81it/s]

Mean Precision: 0.7648
Mean Recall: 0.8146
Mean F1-Score: 0.7807
Metrics saved to Outputs/metrics_easy_num_faces_mtcnn.json
Predictions saved to Outputs/predictions_easy_num_faces_mtcnn.json


In [9]:
json_path = "Data/WIDER FACE Validation Set/wider_face_num_faces_medium.json"
output_path_metrics = "Outputs/metrics_medium_num_faces_mtcnn.json"
output_path_predictions = "Outputs/predictions_medium_num_faces_mtcnn.json"

evaluate_mtcnn_with_predictions(json_path, output_path_metrics, output_path_predictions)

Evaluating Images: 100%|███| 1089/1089 [09:42<00:00,  1.87it/s]

Mean Precision: 0.8388
Mean Recall: 0.7122
Mean F1-Score: 0.7448
Metrics saved to Outputs/metrics_medium_num_faces_mtcnn.json
Predictions saved to Outputs/predictions_medium_num_faces_mtcnn.json


In [12]:
json_path = "Data/WIDER FACE Validation Set/wider_face_num_faces_hard.json"
output_path_metrics = "Outputs/metrics_hard_num_faces_mtcnn.json"
output_path_predictions = "Outputs/predictions_hard_num_faces_mtcnn.json"

evaluate_mtcnn_with_predictions(json_path, output_path_metrics, output_path_predictions)

Evaluating Images: 100%|███| 1015/1015 [10:21<00:00,  1.63it/s]


Mean Precision: 0.8954
Mean Recall: 0.4972
Mean F1-Score: 0.5915
Metrics saved to Outputs/metrics_hard_num_faces_mtcnn.json
Predictions saved to Outputs/predictions_hard_num_faces_mtcnn.json


In [19]:
import random


def evaluate_mtcnn_with_random_subset(
    json_path,
    output_metrics_path,
    predictions_output_path,
    iou_threshold=0.5,
    num_samples=None,
    seed=42
):
    """
    Evaluate MTCNN model on a subset of images from the given JSON annotation file,
    optionally sampling a random subset, and save the model's predictions in a separate JSON file.

    Args:
        json_path (str): Path to the JSON file containing annotations.
        output_metrics_path (str): Path to save the evaluation metrics as a JSON file.
        predictions_output_path (str): Path to save the model predictions as a JSON file.
        iou_threshold (float, optional): IoU threshold to consider a prediction as true positive. Defaults to 0.5.
        num_samples (int, optional): Number of random samples to evaluate. If None, evaluate on all images. Defaults to None.
        seed (int, optional): Random seed for reproducibility when sampling. Defaults to 42.

    Returns:
        None: Prints mean precision, recall, and F1-score, saves metrics and predictions to JSON files.
    """
    # Load the annotations
    try:
        with open(json_path, 'r') as json_file:
            annotations = json.load(json_file)
        logger.info(f"Loaded {len(annotations)} annotations from {json_path}.")
    except Exception as e:
        logger.error(f"Failed to load annotations from {json_path}: {e}")
        return

    # Randomly sample the dataset if num_samples is specified
    if num_samples is not None:
        random.seed(seed)
        sampled_size = min(num_samples, len(annotations))
        annotations = random.sample(annotations, sampled_size)
        logger.info(f"Randomly sampled {sampled_size} annotations with seed {seed}.")

    # Initialize the MTCNN model
    detector = MTCNN()
    logger.info("Initialized MTCNN detector.")

    # Metrics for all images
    precision_scores = []
    recall_scores = []
    f1_scores = []

    # Store predictions
    predictions_data = []

    # Iterate over all image annotations with a progress bar
    for index, image_annotation in enumerate(tqdm(annotations, desc="Evaluating Images")):
        # Extract ground truths and convert to [x1, y1, x2, y2] format
        ground_truths = []
        for gt in image_annotation.get('image_info', []):
            bbox = gt.get('bbox', [])
            if len(bbox) != 4:
                logger.warning(f"Invalid bbox format in image {image_annotation.get('image_path', 'unknown')}. Skipping this ground truth.")
                continue
            x, y, w, h = bbox
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            # Ensure coordinates are positive
            x1, y1, x2, y2 = max(0, x1), max(0, y1), max(0, x2), max(0, y2)
            ground_truths.append([x1, y1, x2, y2])

        # Load the image
        image_path = image_annotation.get('image_path', '')
        if not image_path:
            logger.warning(f"No image path provided for annotation index {index}. Skipping.")
            continue

        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Error: Could not load image at {image_path}. Skipping.")
            continue

        # Convert image from BGR (OpenCV format) to RGB (MTCNN expects RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get predictions using MTCNN
        detections = detector.detect_faces(image_rgb)
        predictions = []
        for face in detections:
            # MTCNN returns 'box' as [x, y, width, height]
            box = face.get('box', [])
            if len(box) != 4:
                logger.warning(f"Invalid detection box format in image {image_path}. Skipping this detection.")
                continue
            x, y, w, h = box
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
            # Ensure coordinates are within image boundaries
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(image.shape[1] - 1, x2), min(image.shape[0] - 1, y2)
            predictions.append([x1, y1, x2, y2])

        # Save predictions for this image
        predictions_data.append({
            "image_path": image_path,
            "predicted_boxes": predictions
        })

        # Calculate metrics
        if not predictions and not ground_truths:
            precision, recall, f1_score = 1.0, 1.0, 1.0  # Perfect score when both are empty
        elif not predictions:
            precision, recall, f1_score = 0.0, 0.0, 0.0
        elif not ground_truths:
            precision, recall, f1_score = 0.0, 0.0, 0.0
        else:
            precision, recall, f1_score = evaluate_predictions(predictions, ground_truths, iou_threshold)

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1_score)

    # Calculate mean metrics
    num_evaluated = len(precision_scores)
    mean_precision = sum(precision_scores) / num_evaluated if num_evaluated else 0
    mean_recall = sum(recall_scores) / num_evaluated if num_evaluated else 0
    mean_f1 = sum(f1_scores) / num_evaluated if num_evaluated else 0

    # Print the results
    logger.info(f"Evaluated {num_evaluated} images.")
    logger.info(f"Mean Precision: {mean_precision:.4f}")
    logger.info(f"Mean Recall: {mean_recall:.4f}")
    logger.info(f"Mean F1-Score: {mean_f1:.4f}")

    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"Mean F1-Score: {mean_f1:.4f}")

    # Save metrics to a JSON file
    output_data = {
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "mean_f1_score": mean_f1,
        "individual_scores": {
            "precision_scores": [float(p) for p in precision_scores],
            "recall_scores": [float(r) for r in recall_scores],
            "f1_scores": [float(f) for f in f1_scores]
        },
        "predictions": predictions_data
    }

    try:
        with open(output_metrics_path, 'w') as output_file:
            json.dump(output_data, output_file, indent=4)
        logger.info(f"Metrics saved to {output_metrics_path}")
        print(f"Metrics saved to {output_metrics_path}")
    except Exception as e:
        logger.error(f"Error saving metrics to {output_metrics_path}: {e}")
        print(f"Error saving metrics to {output_metrics_path}: {e}")

    # Save predictions to a separate JSON file
    try:
        with open(predictions_output_path, 'w') as pred_file:
            json.dump(predictions_data, pred_file, indent=4)
        logger.info(f"Predictions saved to {predictions_output_path}")
        print(f"Predictions saved to {predictions_output_path}")
    except Exception as e:
        logger.error(f"Error saving predictions to {predictions_output_path}: {e}")
        print(f"Error saving predictions to {predictions_output_path}: {e}")

In [21]:
json_path = "Data/WIDER FACE Validation Set/wider_face_easy_blur.json"
output_path_metrics = "Outputs/metrics_easy_blur_mtcnn.json"
output_path_predictions = "Outputs/predictions_easy_blur_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=700)

Evaluating Images: 100%|█████| 700/700 [17:32<00:00,  1.50s/it]


Mean Precision: 0.8646
Mean Recall: 0.8900
Mean F1-Score: 0.8659
Metrics saved to Outputs/metrics_easy_blur_mtcnn.json
Predictions saved to Outputs/predictions_easy_blur_mtcnn.json


In [23]:
json_path = "Data/WIDER FACE Validation Set/wider_face_medium_blur.json"
output_path_metrics = "Outputs/metrics_medium_blur_mtcnn.json"
output_path_predictions = "Outputs/predictions_medium_blur_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=700)

Evaluating Images: 100%|█████| 700/700 [17:42<00:00,  1.52s/it]


Mean Precision: 0.8229
Mean Recall: 0.7554
Mean F1-Score: 0.7718
Metrics saved to Outputs/metrics_medium_blur_mtcnn.json
Predictions saved to Outputs/predictions_medium_blur_mtcnn.json


In [24]:
json_path = "Data/WIDER FACE Validation Set/wider_face_hard_blur.json"
output_path_metrics = "Outputs/metrics_hard_blur_mtcnn.json"
output_path_predictions = "Outputs/predictions_hard_blur_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=700)

Evaluating Images: 100%|█████| 700/700 [19:37<00:00,  1.68s/it]


Mean Precision: 0.8111
Mean Recall: 0.4078
Mean F1-Score: 0.5043
Metrics saved to Outputs/metrics_hard_blur_mtcnn.json
Predictions saved to Outputs/predictions_hard_blur_mtcnn.json


In [25]:
json_path = "Data/WIDER FACE Validation Set/wider_face_easy_occlusion.json"
output_path_metrics = "Outputs/metrics_easy_occlusion_mtcnn.json"
output_path_predictions = "Outputs/predictions_easy_occlusion_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=300)

Evaluating Images: 100%|█████| 300/300 [02:48<00:00,  1.78it/s]

Mean Precision: 0.8750
Mean Recall: 0.8600
Mean F1-Score: 0.8536
Metrics saved to Outputs/metrics_easy_occlusion_mtcnn.json
Predictions saved to Outputs/predictions_easy_occlusion_mtcnn.json


In [26]:
json_path = "Data/WIDER FACE Validation Set/wider_face_medium_occlusion.json"
output_path_metrics = "Outputs/metrics_medium_occlusion_mtcnn.json"
output_path_predictions = "Outputs/predictions_medium_occlusion_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=300)

Evaluating Images: 100%|█████| 300/300 [02:29<00:00,  2.00it/s]

Mean Precision: 0.7574
Mean Recall: 0.6025
Mean F1-Score: 0.6401
Metrics saved to Outputs/metrics_medium_occlusion_mtcnn.json
Predictions saved to Outputs/predictions_medium_occlusion_mtcnn.json


In [27]:
json_path = "Data/WIDER FACE Validation Set/wider_face_hard_occlusion.json"
output_path_metrics = "Outputs/metrics_hard_occlusion_mtcnn.json"
output_path_predictions = "Outputs/predictions_hard_occlusion_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=300)

Evaluating Images: 100%|█████| 300/300 [02:51<00:00,  1.74it/s]

Mean Precision: 0.8145
Mean Recall: 0.4126
Mean F1-Score: 0.5098
Metrics saved to Outputs/metrics_hard_occlusion_mtcnn.json
Predictions saved to Outputs/predictions_hard_occlusion_mtcnn.json


In [29]:
json_path = "Data/WIDER FACE Validation Set/wider_face_normal_illumination.json"
output_path_metrics = "Outputs/metrics_normal_illumination_mtcnn.json"
output_path_predictions = "Outputs/predictions_normal_illumination_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions, num_samples=400)

Evaluating Images: 100%|█████| 400/400 [03:21<00:00,  1.98it/s]

Mean Precision: 0.8445
Mean Recall: 0.7231
Mean F1-Score: 0.7462
Metrics saved to Outputs/metrics_normal_illumination_mtcnn.json
Predictions saved to Outputs/predictions_normal_illumination_mtcnn.json


In [30]:
json_path = "Data/WIDER FACE Validation Set/wider_face_extreme_illumination.json"
output_path_metrics = "Outputs/metrics_extreme_illumination_mtcnn.json"
output_path_predictions = "Outputs/predictions_extreme_illumination_mtcnn.json"

evaluate_mtcnn_with_random_subset(json_path, output_path_metrics, output_path_predictions,num_samples=400)

Evaluating Images: 100%|█████| 400/400 [03:46<00:00,  1.76it/s]

Mean Precision: 0.8374
Mean Recall: 0.5526
Mean F1-Score: 0.6140
Metrics saved to Outputs/metrics_extreme_illumination_mtcnn.json
Predictions saved to Outputs/predictions_extreme_illumination_mtcnn.json


In [35]:
def calculate_average_prediction_time_mtcnn(json_path, max_images=100, seed=42):
    """
    Calculate the average prediction time of the MTCNN model on a random subset of images.

    Args:
        json_path (str): Path to the JSON file containing annotations.
        max_images (int): Number of images to process.
        seed (int): Random seed for reproducibility.

    Returns:
        float: The average prediction time per image in seconds.
    """
    # Check if the JSON file exists
    if not os.path.exists(json_path):
        logger.error(f"JSON file does not exist: {json_path}")
        print(f"Error: JSON file does not exist: {json_path}")
        return None

    # Load the annotations
    try:
        with open(json_path, 'r') as json_file:
            annotations = json.load(json_file)
            logger.info(f"Loaded {len(annotations)} annotations from {json_path}")
    except json.JSONDecodeError as e:
        logger.error(f"Error decoding JSON file: {e}")
        print(f"Error decoding JSON file: {e}")
        return None

    # Randomly select a subset of the data
    random.seed(seed)
    if len(annotations) > max_images:
        annotations_subset = random.sample(annotations, max_images)
        logger.info(f"Selected {max_images} random images for evaluation.")
    else:
        annotations_subset = annotations
        logger.warning(f"Dataset contains fewer than {max_images} images. Using all available images ({len(annotations_subset)}).")

    # Initialize the MTCNN model
    detector = MTCNN()
    logger.info("Initialized MTCNN detector.")

    # Measure prediction times
    prediction_times = []

    for image_annotation in tqdm(annotations_subset, desc="Measuring Prediction Times"):
        # Load the image
        image_path = image_annotation.get('image_path', '')
        if not image_path:
            logger.warning("No image path provided for an annotation. Skipping.")
            continue

        if not os.path.exists(image_path):
            logger.error(f"Image file does not exist: {image_path}. Skipping.")
            print(f"Error: Image file does not exist: {image_path}. Skipping.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            logger.error(f"Error: Could not load image at {image_path}. Skipping.")
            print(f"Error: Could not load image at {image_path}. Skipping.")
            continue

        # Convert image from BGR (OpenCV format) to RGB (MTCNN expects RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Start the timer
        start_time = time.time()

        # Make predictions using MTCNN
        detections = detector.detect_faces(image_rgb)

        # End the timer
        end_time = time.time()

        # Calculate prediction time
        prediction_time = end_time - start_time
        prediction_times.append(prediction_time)

    # Calculate the average prediction time
    if prediction_times:
        average_time = sum(prediction_times) / len(prediction_times)
        logger.info(f"Average prediction time: {average_time:.4f} seconds per image")
        print(f"Average prediction time: {average_time:.4f} seconds per image")
        return average_time
    else:
        logger.warning("No valid images were processed.")
        print("No valid images were processed.")
        return None

# Example usage
if __name__ == "__main__":
    json_path = "Data/WIDER FACE Validation Set/wider_face_val_annotations.json"
    calculate_average_prediction_time_mtcnn(json_path, max_images=100, seed=42)

Measuring Prediction Times: 100%|█| 100/100 [00:52<00:00,  1.91

Average prediction time: 0.5005 seconds per image
